# Vergleich zweier Netzwerke

_Seminar »Figurennetzwerke digital analysiert«, Prof. Frank Fischer, Freie Universität Berlin, Sommersemester 2022._

_Alisa Istomina, Irvin Hostettler._

Dieses Workbook vergleicht zwei verschiedene Gephi-Netzwerke und untersucht die Unterschiede in der Gewichtung der Kanten und speichert das Ergebnis wiederum als Netzwerk. Ziel ist es, die relativen Veränderungen von Figurennetzerken verschiedener Bearbeitungen eines Stoffes zu visualisieren.

## Laden

In der Liste werden alle Netzwerke angegeben, welche miteinander verglichen werden sollten.

In [201]:
import networkx as nx

from pathlib import Path

NETWORKS = [
    "networks/book.graphml",
    "networks/film-1999.graphml",
    "networks/film-2010.graphml",
    "networks/partitur.graphml"
]

## Normalisierung

Da die Diskriminierung (im Sinne von »Aufteilung«) der Figureninteratkionen auf unterschiedliche Weise geschehen kann (z.B. nach Kapiteln, Filmszenen usw.), besteht die Möglichkeit, die Gewichte zu normalisieren. Hierfür wird der jeweilige Durchschnitt der Gewichte eines Graphens genommen und mit dem des anderen verglichen.

In [202]:
def weight_avg(graph: nx.Graph) -> float:
    """Calculates the average of all edge weights in a given graph."""
    weights = []
    for edge in graph.edges:
        weights.append(graph.get_edge_data(edge[0], edge[1])["weight"])
    return sum(weights) / len(weights)

def avg_ratio(graph_a: nx.Graph, graph_b: nx.Graph) -> float:
    """Calculates the quotient between the weight averages of the two given graphs."""
    return weight_avg(graph_b) / weight_avg(graph_a)
    

## Vergleichen

Ausgehend vom `graph_a` werden nun alle Edges und deren Gewichte mit den von `graph_b` verglichen.

In [203]:
def compare_graphs(graph_a: nx.Graph, graph_b: nx.Graph) -> nx.Graph:
    """Compare two graphs and return the result in the form of a graph."""
    rsl = nx.Graph()
    avg_rt = avg_ratio(graph_a, graph_b)
    for edge_a in graph_a.edges:
        data_a = graph_a.get_edge_data(edge_a[0], edge_a[1])
        data_b = graph_b.get_edge_data(edge_a[0], edge_a[1])
        if data_b is None:
            value = 1.0
            value_normalized = 1.0
            connection = False
        else:
            value_normalized = round((data_b['weight'] / data_a['weight'] / avg_rt) * 10, 1)
            value = (data_b['weight'] / data_a['weight']) * 10
            connection = True
        rsl.add_edge(edge_a[0], edge_a[1], weight=value, weight_normalized=value_normalized, connection=connection)
    return rsl

## Normalisierung Knotennamen

Da teilweise die Groß- und Kleinschreibung einzelner Knoten voneinander abweicht, werden alle Knotennamen ohne Großbuchstaben wiedergegeben.

In [204]:
import string

def normalize_names(graph: nx.Graph) -> nx.Graph: 
    mapping = {}
    for node in graph.nodes:
        mapping[node] = node.lower()
    return nx.relabel_nodes(graph, mapping)

## CSV Export

Für die Analyse ist es hilfreich, wenn die Daten auch in einer CSV vorliegen. Somit können diese direkt in einer Tabellenkalkultion geöffnet werden.

In [222]:
import csv

def edge_dict_list(graph: nx.Graph):
    """Converts to a list of edges as a dict."""
    edges = nx.to_edgelist(graph)
    rsl = []
    for edge in edges:
        rsl.append({
            "source": edge[0],
            "target": edge[1],
            "weight": edge[2]["weight"],
            "weight_normalized": edge[2]["weight_normalized"],
            "connected": edge[2]["connection"],
        })
    return rsl


def write_csv(graph: nx.Graph, path: Path):
    data = edge_dict_list(graph)
    with open(path, "w") as file:
        writer = csv.DictWriter(file, fieldnames=data[0].keys())
        writer.writeheader()
        for entry in data:
            writer.writerow(entry)

## Ausführen

Alle Netzwerke aus der Liste `NETWORKS` werden miteinander verglichen und das Ergebnis gespeichert.

In [223]:
def compare_files(file_a: Path, file_b: Path, folder: Path):
    """Compares two files with each other and writes the output to a file."""
    rsl = compare_graphs(
        normalize_names(nx.read_graphml(file_a)),
        normalize_names(nx.read_graphml(file_b)),
    )
    nx.write_graphml(rsl, folder / f"{file_a.stem}_{file_b.stem}.graphml")
    write_csv(rsl, folder / f"{file_a.stem}_{file_b.stem}.csv")

def compare_all(files: list[str]):
    """Compare all permutations of the file with each other and save the result."""
    for file_a in NETWORKS:
        for file_b in NETWORKS:
            if file_a == file_b:
                continue
            compare_files(Path(file_a), Path(file_b), Path("comp"))

compare_all(NETWORKS)